In [1]:
using Pkg
Pkg.activate("EnvFerrite")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


# Electrical Impedance Tomography with TV Regularization

## Introduction

In this example we give a basic implementation of of the real valued Calderon problem relevant to Electrical Impedance Tomography. We will generate data and afterwards solve the inverse problem with a numerical solver and implement TV regularization.  

### Forward EIT:
Given a conductivity $\gamma: \Omega\subset\mathbb{R}^N \rightarrow \mathbb{R}^N_{+}$ our solution $u\in H^1(\Omega,\mathbb{R}^N)$ has to confirm to the equation:


$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \forall x \in \Omega $$

For that equation we will choose a set of electrical current patterns 

$$ g_1, ..., g_n \in H^{-\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$

such that:  

$$ \int_{\partial\Omega}g_i \, d\partial\Omega = 0 $$

to inject into the material via Neumann boundary conditions:

$$ \gamma\frac{\partial u_i}{\partial n} =g_i \quad \forall x \in \partial\Omega $$

In order to get the corresponding voltages 
$$ f_1, ..., f_n \in H^{-\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$
we will measure the corresponding voltage as:
$$ f_i := u_i|_{\partial\Omega} $$

with those boundary pairs $ (f_1,g_1), ... , (f_n,g_n) $ we now have an approximation of the Dirichlet to Neumann map:
$$ \Lambda_\gamma: H^{-\frac{1}{2}}(\partial \Omega)\rightarrow H^{-\frac{1}{2}}(\partial \Omega) $$
This is called forward EIT since we just approximated the map:
$$ \gamma \rightarrow  \Lambda_\gamma $$

However real Electrical Impedance Tomography requires us to solve an **inverse Problem** where we have to reconstruct:
$$ \Lambda_\gamma \rightarrow  \gamma $$
for our approximation of $\Lambda_\gamma$ given by voltage-current boundary pairs $(f_i,g_i)$.

### Weak formulation
Given the strong formulation:
$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \text{with Neumann BC:}\quad \gamma\frac{\partial u}{\partial n} = g $$

The weak formulation is:
$$ \int\limits_\Omega \gamma \nabla(u)\cdot\nabla(v) \, d \Omega = \int\limits_{\partial\Omega} g \,v \,d\partial\Omega  \quad \forall v\in H^1(\Omega)$$

### Inverse EIT
We will use $\gamma$ to refer to the true underlying conductivity and $\sigma$ for our current conductivity guess.
We will choose the simplest minimization functional for optimization:
$$ J_i(u_i,\sigma) = \| f_i- u_i\|^2_{\mathcal{L}^2(\partial\Omega)} = \int_{\partial\Omega}(f_i-u_i)^2 \,d\partial\Omega $$
Such that our problem becomes:
$$ \underset{\sigma}{\min} \sum\limits_{i=1}^n J_i(u_i,\sigma) $$
such that:
$$ \nabla u_i\cdot(\sigma\nabla u_i) = 0\quad  \text{and Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i \quad \forall i\in\{1, ...,n\}$$
Given the problem our lagrangian becomes:
$$ \mathcal{L}(\sigma, u, \lambda) = \sum\limits_{i=1}^n\left( J_i(u_i,\sigma) + \langle \lambda_i, \nabla\cdot(\sigma\nabla u_i) \rangle_{\mathcal{L}^2(\Omega)}   \right) $$

from this we will use [Adjoint state methods](https://en.wikipedia.org/wiki/Adjoint_state_method#General_case) to calculate the gradient.
Without stating any of the steps of the derivation we end up with:
- State Equation (Variation with $\delta_\lambda$)
$$ \nabla \cdot(\sigma\nabla u_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i$$

- Adjoint Equation (Variation with $\delta_u$)
$$ \nabla \cdot(\sigma\nabla\lambda_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial \lambda_i}{\partial n} = 2(u_i-f_i)$$
- Functional Derivative (Variation with $\delta_\sigma$)
$$ \delta_i\sigma = -\nabla u_i \cdot \nabla \lambda_i $$

#### Total Variation (TV) regularization
for real world examples and numerical stability we have to assume that our system contains some noise, like $f = f_{true} + \epsilon$. Since the inverse EIT problem is highly ill conditioned we have to consider regularization.


Because $|\nabla\sigma|$ is non-differentiable when $\nabla\sigma=0$, we use
$$
\mathcal{R}_{TV}(\sigma) = \int_\Omega \sqrt{|\nabla\sigma|^2+\eta}\, d\Omega,
$$
and the gradient:
$$ \delta_\sigma\mathcal{R}= -\nabla\cdot \left(   \frac{\nabla\sigma}{\sqrt{|\nabla \sigma|^2+ \eta}} \right) $$
with a small $\eta$ to revent division by zero.
This is a $L^2$ projection that requires us to sove the weak form.
##### Weak formulation of TV Regularizer:
$$ \int_\Omega wv \,d\Omega = \int_\Omega \frac{\nabla(\sigma)}{\sqrt{|\nabla\sigma|^2+\eta}}\cdot\nabla(v)\, d\Omega \quad \forall v\in FESpace$$


### Full reconstruction Algorithm (Conceptual)
+ Preallocate Massmatrix M and L^2 projector (Cholesky factorization)
+ Start with conductivity guess $\sigma_0$ (In our case: $\sigma_0(x) = 1.0$)
+ Preallocate & initialize Conjugate Gradient(CG) solver for $u_1, ...,u_n.\lambda_1,.., \lambda_n,w$.
+ Repeat till tolerance is reached or other stopping condition:
    + From $\sigma_t$ assemble $K_{\sigma_t}$
    + for all $i = 1, ...,n$ (in parallel)
        + Calculate $u_i$ (State equation) as well as the $\mathcal{L}^2(\partial\Omega)$ error: $\delta u_i$
        + Calculate $\lambda_i$ (Adjoint equation)
        + Calculate $\delta\sigma_i$ (Functional derivative)
    + Calculate TV Regularization gradient and error.
    + Update $\sigma_{t+1} = \sigma_t +\beta\, \delta_{TV}\sigma + \sum_{i=1}^n \alpha_i\,\delta_i\sigma $ (with Gauss-Newton with Levenberg-Marquardt).

## Implementation
### Preliminaries
Obvious Imports:

In [2]:
using Ferrite
using SparseArrays
using LinearAlgebra
using Revise
using Interpolations
using Plots
using Statistics

For simplicity we will use a quadratic grid with quadrilateral elements. We are using Quadrilaterals for now:

In [ ]:
grid = generate_grid(Quadrilateral, (16, 16));
dim = Ferrite.getspatialdim(grid)
order = 1

ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(2)
qr_face = FacetQuadratureRule{RefQuadrilateral}(2)
cellvalues = CellValues(qr, ip)
facetvalues = FacetValues(qr_face, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)



proj = L2Projector(ip,grid)
#cells = 1:getncells(grid)
#add!(proj, collect(cells), Lagrange{RefQuadrilateral, 1}(); qr_rhs = qr)
#close!(proj)

∂Ω = union(getfacetset.((grid,), ["left", "top", "right", "bottom"])...)
length(∂Ω)

64

For later use we will assemble and cholesky decompose the mass matrix once.

In [4]:
# This is supposed to be: ∫(u*v)dΩ
function assemble_M(cellvalues::CellValues,dh::DofHandler)
    M = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Me = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Me, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)      
            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    φⱼ = shape_value(cellvalues, q_point, j)
                    Me[i,j] += φᵢ * φⱼ * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Me)
    end
    return M
end   

M = assemble_M(cellvalues,dh)
MC = cholesky(M)

SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  3815
nnz:     3815
success: true


Furthermore we want to know which entries of the force vector correspond to the boundary:
We will get:
- the count of nonzero entries in the force vector
- the position of non zero entries
- a function "up" to cast a vector of length of boundary dofs into the length of the force vector
- a function "down" to cast a vector into the length of the dofs of the force vector that lay on the boundary.

In [5]:
function produce_nonzero_positions(v, atol=1e-8, rtol=1e-5)
    approx_zero(x; atol=atol, rtol=rtol) = isapprox(x, 0; atol=atol, rtol=rtol)
    non_zero_count = count(x -> !approx_zero(x), v)
    non_zero_positions = zeros(Int, non_zero_count)
    non_zero_indices = findall(x -> !approx_zero(x), v)
    g_down = (x) -> x[non_zero_indices]
    g_up = (x) -> begin
        v = zeros(eltype(x), length(v))
        v[non_zero_indices] = x
        return v
    end
    return non_zero_count, non_zero_positions, g_down, g_up
end
function produce_nonzero_positions(facetvalues::FacetValues, dh::DofHandler, ∂Ω)
    f = zeros(ndofs(dh))
        for facet in FacetIterator(dh, ∂Ω)
        fe = zeros(ndofs_per_cell(dh))
        reinit!(facetvalues, facet)
        for q_point in 1:getnquadpoints(facetvalues)
            dΓ = getdetJdV(facetvalues, q_point)            
            for i in 1:getnbasefunctions(facetvalues)
                δu = shape_value(facetvalues, q_point, i)
                fe[i] += δu * dΓ
            end
        end
        assemble!(f, celldofs(facet), fe)
    end
     nzc, nzpos, g_down, g_up = produce_nonzero_positions(f)
     return  nzc, nzpos, g_down, g_up, f
end

nzc,nzpos, down, up = produce_nonzero_positions(facetvalues, dh,∂Ω)
@assert nzc == length(∂Ω)  # This is not true in Gridap.jl

In [6]:
## Sanity check:
# I have never questioned the assumption that up∘down == id and down∘up == id. Maybe I should check this.
test_vec = [i for i in 1:nzc]
@assert down(up(test_vec)) == test_vec
@assert up(down(up(test_vec))) == up(test_vec)

### Data generation

Now we will make up some conductivity. As well as some current patterns:

In [7]:
conductivity  = (x) -> 1.1 + sin(x[1]) * cos(x[2])

#22 (generic function with 1 method)

For later we want to project that function unto Q1 FE space for that we want to 

<span style="color:red">
I am somewhat clueless on how to actually project the function conductivity unto a vector living in the FE space.
The example as stated in the documentation https://ferrite-fem.github.io/Ferrite.jl/stable/reference/export/#Projection-of-quadrature-point-data seems to now work. 
Below I try to assemble a function projector as described in the domentation. it fails for now.


</span>

In [ ]:
function project_function(f,grid)
#proj = L2Projector(ip,grid)
    quad_set = collect(1:getncells(grid))
    tria_set = quad_set
    proj = L2Projector(grid)
    qr_quad = QuadratureRule{RefQuadrilateral}(2)
    add!(proj, quad_set, Lagrange{RefQuadrilateral, 1}(); qr_rhs = qr_quad)
    close!(proj)

    vals = Dict{Int, Vector{Float64}}() # Can also be Vector{Vector},
                                    # indexed with cellnr
    for (set, qr) in ((quad_set, qr_quad))
        nqp = getnquadpoints(qr)
        for cellnr in set
            vals[cellnr] = rand(nqp)
        end
    end

    projected = project(proj, vals)
return projected
end
γ = project_function(conductivity,grid)

project_function (generic function with 1 method)

In [ ]:
# This function assembles the stiffness matrix from a given vector.
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ::AbstractVector)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            # How do I get the index x of the vector γ?
            σ = γ[cellid(cell)]
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q_point, i)
                #u = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += σ* (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_from_vec = assemble_K(cellvalues, dh,γ)

289×289 SparseMatrixCSC{Float64, Int64} with 2401 stored entries:
⎡⠻⣦⡀⠀⠐⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠘⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠰⣤⣀⠈⠻⣦⡘⢶⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠉⠛⢲⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣌⠻⢆⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠳⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠱⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢶⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢳⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⢆⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠳⣌⠻⣦⡙⢦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠱⣦⡙⢦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⣦⡙⢦⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣌⠻⢆⡙⠦⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⡌⠻⣦⎦

In [10]:
# This is matrix assembly on a function. How do I do it if the conductivity is given as a coefficient vector for Q1 FE Space?
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            x = spatial_coordinate(cellvalues, q_point, getcoordinates(cell))
            σ = γ(x)
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += σ * (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_true = assemble_K(cellvalues, dh, conductivity)
K_true_LU = lu(K_true)

SparseArrays.UMFPACK.UmfpackLU{Float64, Int64}
L factor:
289×289 SparseMatrixCSC{Float64, Int64} with 3815 stored entries:
⎡⣳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠉⠉⠱⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡶⠿⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡀⠀⢀⣨⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠉⠁⠈⠉⠉⠉⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠳⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣨⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣠⣼⠐⢻⣿⠛⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠷⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠾⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣤⣤⣾⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠃⠀⠒⠋⠈⠛⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠄⠴⠷⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⠀⣠⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡀⢠⣤⡄⠀⣤⣼⣿⠈⢙⣼⡿⠈⢿⣷⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⣀⣰⣾⠇⠀⠸⠇⠀⠿⣿⡁⠀⠀⠀⠀⢈⣹⣿⠀⠀⠀⠐⠟⠿⣿⣿⣷⣄⠀⠀⠀⠀⎥
⎢⣶⢶⡆⠀⠀⢀⡀⣀⣹⣿⣉⠀⠀⠀⢀⠀⣀⣉⠃⠀⠀⠲⠶⠾⠿⠀⠀⠀⠀⠀⠀⠀⠀⠿⣿⣿⣷⣄⠀⠀⎥
⎣⠛⠚⠃⠒⠒⢻⣥⣽⣇⣉⠀⠀⠀⠀⠠⣤⣤⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣤⣬⣽⣿⣷⣄⎦
U factor:
289×289

In [41]:
# Implement a sanity check if the two matrices assembled from the function and the vector are roughly the same (use relatively coarse ≈ )

In [11]:
# Sanity check: (positive semidefinite self adjoint stiffness matrix)
@assert K_true == K_true'  # Not true in Gridap.jl
if ndofs(dh) < 500
    K_dense = Matrix(K_true)
    eig_min = minimum(eigvals(K_dense))
    @assert eig_min > -1e-14
    print("Smallest eigenvalue: ", eig_min)
end

Smallest eigenvalue: -1.94652976991602e-16

Now we generate current patterns: We assume one current source and one current sink. Important is that it sums of to zero.
We generate the right hand side force vectors $g_1, ... g_n$ as an Matrix G and calculate $f_1, ..., f_n$

In [12]:
num_modes = Int64((nzc^2-nzc)//2)
G_small = zeros(nzc,num_modes)
G = zeros(ndofs(dh),num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        G_small[i,k] = 1.0
        G_small[j,k] = -1.0
        G[:,k] = up(G_small[:,k])
        k += 1
    end
end
F_big = K_true_LU \ G
F = zeros(nzc,num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        F[:,k] = down(F_big[:,k])
        k += 1
    end
end
col_means = mean(F, dims=1)
F .-= col_means

64×2016 Matrix{Float64}:
  2.30292     2.26394      2.90952     3.30183    …   0.0163961   0.00897954
 -0.812527    0.109923     0.497171    0.924775       0.0163502   0.0089682
  0.148901   -0.980469     0.710295    1.0545         0.0164435   0.00899124
 -0.109424    0.0647217   -0.997809    0.143253       0.0162173   0.00893539
 -0.0741391   0.016607    -0.249065   -1.11555        0.0160037   0.00888259
 -0.0512821   0.00063017  -0.162015   -0.350418   …   0.0157158   0.00881137
 -0.0422296  -0.00745448  -0.118149   -0.239163       0.0153609   0.00872349
 -0.0374468  -0.011809    -0.0976012  -0.181881       0.0149479   0.00862104
 -0.0347296  -0.0143707   -0.0861511  -0.153069       0.0144873   0.00850662
 -0.0330712  -0.0159639   -0.0793021  -0.13635        0.0139928   0.00838351
  ⋮                                               ⋱               ⋮
 -0.0276973  -0.0213465   -0.0578673  -0.0872506  …   0.0356959   0.0136314
 -0.027882   -0.0211539   -0.0585805  -0.0887961      0.040147

To be realistic we will add some noise:
We have to ensure that our noise has mean zero.

In [13]:
function mean_zero_noise(n::Int64, σ::Float64)
    out = σ * randn(n)
    mean = Statistics.mean(out)
    out .- mean
end

mean_zero_noise (generic function with 1 method)

To simplify things we can also do SVD:

In [ ]:
# Implement SVD here later
# reduce the number of modes according to used SVD modes

Here we define a struct where we save and preallocate all the necessary information for the solver step.

In [15]:
mutable struct EITMode
    u::AbstractVector
    λ::AbstractVector
    δσ::AbstractVector
    f::AbstractVector
    g::AbstractVector
    error::Float64
    length::Int64
    m::Int64
end
function EITMode(g::AbstractVector, f::AbstractVector)
    L = length(g)
    M = length(f)
    zerosL = zeros(L)
    return EITMode(zerosL, zerosL, zerosL, f, g, 0.0, L, M)
end
mode_dict = Dict()
for i in 1:num_modes
    mode_dict[i] = EITMode(G[:,i],F[:,i])
end

### Solving EIT

We will now assume a starting conductivity guess $\sigma_0(x) = 1.0 $

In [16]:
σ₀ = (x) -> 1.0

#24 (generic function with 1 method)

We would prefer to save $\sigma$ as a vector for use in FEM and also have a method to export each

In [42]:
# Project function here: 
#return vector.

A prerequisite is that we calculate the Tensor $\nabla(u)\cdot\nabla(\lambda)$


In [ ]:
# This function is supposed to calculate ∇(u)⋅∇(λ)
# Assemble right hand side:
function calculate_bilinear_form_rhs!
# code here
# Assemble

function calculate_bilinear_form!(mode, ...)

    calculate_bilinear_form_rhs!(mode, ...)
    # Code here
    # Either: 
    mode.δ = project(proj,vals)
    # or
    mode.δ = MC \ mode.rhs # Cholesky decompostion of Mass matrix.
end

With the given matrix and projector our we need to optimize for every mode $(f_i,g_i)$:

In [ ]:
function state_adjoint_step!(mode::EITMode, K::AbstractMatrix,proj, M::AbstractMatrix, maxiter=500)
    cg!(mode.u,K, mode.g; maxiter = maxiter)
    b = down(mode.u)
    b = 2*(b - mode.f)
    b .-= Statistics.mean(b)
    cg!(mode.λ, K, up(b); maxiter = maxiter)
    mode.error = norm(b)^2
    # add calculation of ∇(u)⋅∇(λ) here once figured out
    mode.δ = calculate_bilinear_form(mode)    
    # Check whether this needs + or - as a sign.
end

state_adjoint_step! (generic function with 2 methods)

Additinal we need to assemble the TV regularizer. The required Mass matrix we already have asembled and ready to use. 


In [18]:
mutable struct TV
    δ::AbstractArray # Is supposed to hold the error
    rhs::AbstractArray # 
    err_vec::AbstractArray
    error::Float64
end
function TV(n::Int64)
    TV(zeros(n),zeros(n),zeros(n),0.0)
end
function calc_tv_step!(σ::AbstractVector{Float64},tv::TV, dh::DofHandler,cellvalues::CellValues,M::AbstractMatrix, η=1e-8)
    n = ndofs(dh)
    rhs = TV.rhs

    a = TV.err_vec
    TV.rhs = zeros(n)

    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        dofs = celldofs(cell)
        σe = σ[dofs]

        re = zeros(length(dofs))
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)

            ∇ϕ = shape_gradient(cellvalues, q_point)
            ∇σ = ∇ϕ * σe
            a = 1 / sqrt(dot(∇σ, ∇σ) + η)
            for (i, ϕi) in enumerate(shape_function_values(cellvalues, q_point))
                ∇ϕi = ∇ϕ[:, i]
                re[i] += a * dot(∇σ, ∇ϕi) * dΩ
            end
        end
        assemble!(r, dofs, re)
    end

    TV.δ = M \ r  
    return w
end

calc_tv_step! (generic function with 2 methods)

For finding suitable stepsizes we will use Gauss-Newton.
Since we want to avoid implementing a 

In [19]:
function gauss_newton(J::Matrix{Float64}, r::Vector{Float64}; λ::Float64=1e-3)
    U, Σ, V = svd(J, full=false)
    n = length(Σ)
    Σ_damped = zeros(n)
    for i in 1:n
        Σ_damped[i] = Σ[i] / (Σ[i]^2 + λ) # Levenberg-Marquardt regularization
    end
    V * (Σ_damped .* (U' * r))
end

gauss_newton (generic function with 1 method)

Now that we have all the pieces we can assemble the full optimization step:

In [ ]:
function full_step!()
    # Assemble Matrix: (from vector)
    K = assemble_K(cell_values,dh,σ)

    J = zeros(num_modes,ndofs(dh))
    r = zeros(num_modes+1)
    # Launch TV regularizer:
    tv_task = Threads.@spawn begin
        calc_TV_step!(σ,tv, dh,cellvalues,M)
    end
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K)
    end
    # Fetch TV regularization
    fetch(tv_task)

    # Fetch gradients & errors
    for i in num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    J[num_modes+1,:] = tv.δ
    r[num_modes+1] = β * tv.error 
    
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    σ = max.(σ+δσ ,1e-12) # Ensure positivity
end

full_step! (generic function with 1 method)

Let's run this optimization loop a few times:

In [43]:
for i in 1:100
    
end

## Plotting


In [49]:
# project is to grid and plot with Plots.jl
# I wanna use Plots.jl and not Makie.jl or similar because lateron i want to implement a NN on the grid as a regularizer using Lux.jl

In [48]:
# Project to dictionary (coordinate,value)
#PointEvalHandler(grid, points::AbstractVector{Vec{dim,T}})
# Put values from dictionary into Array.

This is our original and final reconstruction:

In [47]:
# Plot reconstruction and original with Plots.jl here